## Analyses scripts for the manuscript "The Differential Impact of Climate Interventions along the Political Divide in 60 Countries"

### Install Libraries

In [1]:
# Core analysis 
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import anova
#import bff
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

Autosaving every 30 seconds


### Import data and run data preprocessing

In [2]:
df = pd.read_excel("data63.xlsx")
df

,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,Edu,Income,Indirect_SES,MacArthur_SES,PerceivedSciConsensu_1,Intro_Timer,condition_time_total,WEPTcc,BELIEFcc,POLICYcc
0,R_1r7Yr7R1xMimIvj,usa,5,CollectAction,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,98.0,77.0,86.0,93.0,71.0,...,3.0,5.0,"2,4,5",7.0,80.0,6.056,95.501,1,88.50,89.000000
1,R_2ATgV5VxewrTRiI,usa,1,Control,PolicySupport|BeliefinAnthrCC|Socialmediashari...,49.0,50.0,51.0,51.0,1.0,...,3.0,4.0,"1,2,3,4,5,6,7",5.0,51.0,16.284,200.742,0,50.25,66.222222
2,R_3ENlePBlLyYjExr,usa,9,LetterFutureGen,SocialmediasharingPipedText|PolicySupport|Beli...,87.0,82.0,84.0,83.0,84.0,...,3.0,5.0,"1,2,3,6,7",8.0,73.0,5.701,218.442,1,84.00,84.222222
3,R_2c0HQgvYzas6mus,usa,4,SciConsens,PolicySupport|SocialmediasharingPipedText|Beli...,1.0,1.0,0.0,0.0,1.0,...,3.0,5.0,"1,2,3,4,5,6,7",4.0,1.0,13.282,12.702,0,0.50,1.222222
4,R_00siryCwgdMus6J,usa,8,PluralIgnorance,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,56.0,59.0,61.0,54.0,79.0,...,3.0,5.0,"1,3,5,6,7",7.0,65.0,8.474,19.315,0,57.50,80.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59435,R_2TuXye2HP6EO5KZ,vietnam,11,FutureSelfCont,PolicySupport|SocialmediasharingPipedText|Beli...,49.0,100.0,100.0,74.0,70.0,...,3.0,2.0,"1,2,3,4,5,6,7",7.0,60.0,27.732,873.538,0,80.75,82.500000
59436,R_3R3p8C0gzLKTcCu,vietnam,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,94.0,98.0,98.0,96.0,92.0,...,3.0,2.0,"1,2,4,6,7",3.0,74.0,49.502,0.000,0,96.50,86.444444
59437,R_OeT1JV3yC1aJ4EF,vietnam,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,100.0,100.0,100.0,100.0,80.0,...,3.0,1.0,"2,4,5,6,7",6.0,100.0,7.920,89.072,7,100.00,83.333333
59438,R_2azVf0KXD3rtly8,vietnam,3,NegativeEmotions,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,79.0,51.0,78.0,51.0,0.0,...,3.0,NaN,"1,2,3,4,5,6,7",7.0,100.0,40.222,398.193,8,64.75,66.666667


In [3]:
# Per-country median split
df['ide'] = (df['Politics2_1'] + df['Politics2_9'])/2

new = []
for _, sub in df.groupby("Country"):
    sub.loc[:, "Ideology"] = (sub.loc[:, "ide"] > sub.loc[:, "ide"].median()).astype(float)
    # Ensure NaN ideology when Politics2_1 is NaN
    sub.loc[sub.Politics2_1.isna(), "Ideology"] = np.NaN
    new.append(sub)
df = pd.concat(new)
# Replace 1 with "Conservative" and 0 with "Liberal"
df['Ideology'] = df['Ideology'].replace(1, 'Conservative').replace(0, 'Liberal')

/opt/miniconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [4]:
#transform the datafram from wide format to long format

df_z = pd.melt(
    df.loc[:, ['ResponseId', "Country", "condName", "cond", "Ideology","ide", "BELIEFcc", "WEPTcc"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond"],
    var_name="Type",
    value_name="score"
)
df_z_control=df_z.query('cond==1')
df_control = df.query('cond==1')
#df_control.to_csv('df_control.csv', index=False)

df_control


,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,Indirect_SES,MacArthur_SES,PerceivedSciConsensu_1,Intro_Timer,condition_time_total,WEPTcc,BELIEFcc,POLICYcc,ide,Ideology
5061,R_3ndh8CRntlGPnch,algeria,1,Control,SocialmediasharingPipedText|PolicySupport|Beli...,29.0,32.0,31.0,28.0,31.0,...,"1,2,4,5,6,7",7.0,49.0,5.306,106.759,8,30.00,38.111111,50.0,Liberal
5072,R_3HM8bOnE6bQLcPs,algeria,1,Control,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,14.0,23.0,8.0,46.0,84.0,...,"1,2,4,5,6,7",6.0,77.0,49.231,345.303,8,22.75,89.888889,15.0,Liberal
5083,R_1GP0L143l7yV8M1,algeria,1,Control,PolicySupport|SocialmediasharingPipedText|Beli...,100.0,100.0,53.0,61.0,50.0,...,5,5.0,40.0,22.657,345.559,7,78.50,61.111111,56.0,Liberal
5087,R_3ISScDiQQ7KxFvK,algeria,1,Control,BeliefinAnthrCC|PolicySupport|Socialmediashari...,18.0,13.0,15.0,14.0,82.0,...,"1,2,4,6,7",8.0,13.0,18.616,174.585,4,15.00,65.333333,14.0,Liberal
5100,R_21ng6VSaDWS2muh,algeria,1,Control,BeliefinAnthrCC|PolicySupport|Socialmediashari...,49.0,100.0,100.0,49.0,40.0,...,"1,2,4,5,6,7",5.0,77.0,28.108,290.542,8,74.50,76.666667,NaN,Liberal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59399,R_1db1dsU9anX3Iwf,vietnam,1,Control,PolicySupport|BeliefinAnthrCC|Socialmediashari...,72.0,100.0,100.0,100.0,50.0,...,"1,2,4,5,6,7",7.0,80.0,5.195,189.709,8,93.00,82.888889,10.5,Liberal
59416,R_3PFQCv3jEvnT5RD,vietnam,1,Control,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,92.0,95.0,94.0,93.0,76.0,...,"1,2,3,4,5,6,7",7.0,90.0,11.888,206.495,8,93.50,79.222222,47.0,Liberal
59417,R_2Qgs8q8dUPLjhcA,vietnam,1,Control,PolicySupport|SocialmediasharingPipedText|Beli...,89.0,100.0,100.0,100.0,35.0,...,NaN,NaN,NaN,6.190,166.433,2,97.25,71.111111,6.5,Liberal
59433,R_3GwXA80Oate9obo,vietnam,1,Control,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,100.0,100.0,100.0,100.0,100.0,...,"1,2,3,4,5,6,7",4.0,100.0,4.966,42.906,4,100.00,100.000000,100.0,Conservative


## Main manuscript analyses

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Load R in Python
%load_ext rpy2.ipython

In [6]:
%R library(lme4)
%R library(lmerTest)
%R library(ordinal)


Loading required package: Matrix



Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step




Attaching package: ‘ordinal’

The following objects are masked from ‘package:lme4’:

    ranef, VarCorr



# 1.1 Climate Belief

In [7]:
dfc_belief = pd.melt(
    df_control.loc[:, ['ResponseId', 'Country', 'condName', 'ide','Ideology','Income','Edu','Age','Gender', "WEPTcc",'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'ide','Ideology','Income','Edu','Age','Gender', 'WEPTcc'],
    var_name="Item",
    value_name="Belief"
)
dfc_belief['condName'] = dfc_belief['condName'].replace(['Control'], 'aControl')
dfc_belief = dfc_belief.dropna()
#dfc_belief.to_csv('dfc_belief.csv', index=False)

%Rpush dfc_belief


In [8]:
%%R

M1 <- lmer(Belief ~ ide + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ ide + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 127636.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2602 -0.2858  0.0910  0.3205  5.9168 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 503.7442 22.4442 
 Country    (Intercept)  28.4839  5.3370 
 Item       (Intercept)   0.2646  0.5144 
 Residual               148.6223 12.1911 
Number of obs: 14990, groups:  ResponseId, 3750; Country, 60; Item, 4

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   89.89036    1.22957  220.19202   73.11   <2e-16 ***
ide           -0.18865    0.01644 3713.86092  -11.47   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
ide -0.708


In [8]:
%%R

M1 <- lmer(Belief ~ ide + Income + Edu + Age + as.factor(Gender) + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ ide + Income + Edu + Age + as.factor(Gender) + (1 |  
    Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 127583.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2593 -0.2871  0.0916  0.3222  5.9224 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 497.1486 22.2968 
 Country    (Intercept)  27.0312  5.1992 
 Item       (Intercept)   0.2646  0.5144 
 Residual               148.6204 12.1910 
Number of obs: 14990, groups:  ResponseId, 3750; Country, 60; Item, 4

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)          83.81629    2.42819 1837.36238  34.518  < 2e-16 ***
ide                  -0.17838    0.01653 3701.98706 -10.792  < 2e-16 ***
Income                0.44526    0.22799 2694.06155   1.953  0.05092 .  
Edu                   1.63485

In [11]:
%%R

M1 <- lmer(Belief ~ Ideology + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ Ideology + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 127694.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2228 -0.2889  0.0913  0.3204  5.8909 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 513.4319 22.6590 
 Country    (Intercept)  28.0971  5.3007 
 Item       (Intercept)   0.2645  0.5143 
 Residual               148.6227 12.1911 
Number of obs: 14990, groups:  ResponseId, 3750; Country, 60; Item, 4

Fixed effects:
                 Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)       76.7777     0.9486   85.0663  80.934  < 2e-16 ***
IdeologyLiberal    6.1925     0.7700 3719.2450   8.042 1.17e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
IdeolgyLbrl -0.408


# 1.2 Climate Policy Support

In [9]:
dfc_policy = pd.melt(
    df_control.loc[:, ['ResponseId', 'Country', 'condName', 'Income', 'Edu', 'Age', 'Gender', 'ide','Ideology', "WEPTcc",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'Income', 'Edu', 'Age', 'Gender', 'ide','Ideology', 'WEPTcc'],
    var_name="Item",
    value_name="Policy"
)
dfc_policy = dfc_policy.dropna()
#dfc_policy.to_csv('dfc_policy.csv', index=False)

%Rpush dfc_policy

In [10]:
%%R

M1 <- lmer(Policy ~ ide + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ ide + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 299030.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5864 -0.5263  0.0630  0.6263  4.2149 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 315.86   17.772  
 Country    (Intercept)  18.95    4.354  
 Item       (Intercept) 147.04   12.126  
 Residual               407.27   20.181  
Number of obs: 32915, groups:  ResponseId, 3742; Country, 60; Item, 9

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   76.25396    4.16002    8.95961   18.33 2.07e-08 ***
ide           -0.11015    0.01347 3694.69950   -8.18 3.87e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
ide -0.171


In [14]:
%%R

M1 <- lmer(Policy ~ ide + Income + Edu + Age + as.factor(Gender) + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ ide + Income + Edu + Age + as.factor(Gender) + (1 |  
    Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 298996.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5986 -0.5258  0.0633  0.6275  4.2196 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 313.08   17.694  
 Country    (Intercept)  17.76    4.215  
 Item       (Intercept) 147.05   12.127  
 Residual               407.27   20.181  
Number of obs: 32915, groups:  ResponseId, 3742; Country, 60; Item, 9

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)          68.07553    4.50276   12.29801  15.119 2.59e-09 ***
ide                  -0.10380    0.01357 3677.96422  -7.650 2.55e-14 ***
Income                0.18805    0.18764 2623.65996   1.002   0.3163    
Edu                   2.27483

In [17]:
%%R

M1 <- lmer(Policy ~ Ideology + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ Ideology + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 299061.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5841 -0.5261  0.0628  0.6264  4.2447 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 319.99   17.888  
 Country    (Intercept)  17.57    4.192  
 Item       (Intercept) 147.03   12.126  
 Residual               407.26   20.181  
Number of obs: 32915, groups:  ResponseId, 3742; Country, 60; Item, 9

Fixed effects:
                Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)       68.731      4.108    8.519  16.733 8.19e-08 ***
IdeologyLiberal    3.325      0.629 3699.702   5.285 1.33e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
IdeolgyLbrl -0.077


# 1.3 Behavioral Climate Polarization

In [12]:
dfc_WEPT = df_control[["WEPTcc", "ide", "Ideology", "Income", "Edu", "Age", "Gender", "Country"]]
#dfc_WEPT.to_csv('dfc_WEPT.csv', index=False)

%Rpush dfc_WEPT

In [13]:
%%R

M <- clmm(as.factor(WEPTcc) ~ ide + (1|Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ ide + (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 4214 -6658.28 13324.57 200(608) 6.58e-03 2.1e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1931   0.4394  
Number of groups:  Country 60 

Coefficients:
      Estimate Std. Error z value Pr(>|z|)
ide -0.0008047  0.0012953  -0.621    0.534

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -1.421568   0.097793  -14.54
spacing      0.185666   0.004757   39.03
(872 observations deleted due to missingness)


In [21]:
%%R

M <- clmm(as.factor(WEPTcc) ~ ide + Income + Edu + Age + as.factor(Gender) + (1|Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ ide + Income + Edu + Age + as.factor(Gender) +  
    (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC      niter     max.grad cond.H 
 logit equidistant 3798 -5847.04 11714.08 926(2821) 2.23e-02 3.2e+05

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2266   0.476   
Number of groups:  Country 60 

Coefficients:
                       Estimate Std. Error z value Pr(>|z|)    
ide                  -0.0008018  0.0013962  -0.574   0.5658    
Income                0.0414739  0.0190757   2.174   0.0297 *  
Edu                   0.0330508  0.0513795   0.643   0.5200    
Age                   0.0298260  0.0023364  12.766   <2e-16 ***
as.factor(Gender)2    0.3285394  0.0653664   5.026    5e-07 ***
as.factor(Gender)4    0.7479877  0.3872357   1.932   0.0534 .  
as.factor(Gender)NaN -1.1464303  0.5035319  -2.277   0.0228 *  
---
Signif. codes:  0 ‘***’

In addition: Warning message:
Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead. 


In [25]:
%%R

M <- clmm(as.factor(WEPTcc) ~ Ideology + (1|Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ Ideology + (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 4302 -6816.30 13640.61 142(434) 1.27e-03 8.3e+02

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1987   0.4458  
Number of groups:  Country 60 

Coefficients:
                Estimate Std. Error z value Pr(>|z|)
IdeologyLiberal -0.04918    0.05865  -0.839    0.402

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -1.403479   0.077105  -18.20
spacing      0.186733   0.004719   39.57
(784 observations deleted due to missingness)


In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['BELIEFcc', 'WEPTcc']] = scaler.fit_transform(df[['BELIEFcc', 'WEPTcc']])

df_z = pd.melt(
    df.loc[:, ['ResponseId', "Country", "condName", "cond", "Ideology","ide", "BELIEFcc", "WEPTcc"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond"],
    var_name="Type",
    value_name="score"
)

df_z_control=df_z.query('cond==1')
#df_z_control.to_csv('df_z_control.csv', index=False)

In [27]:
%Rpush df_z_control

In [28]:
%%R

M1 <- lmer(score ~ Type*ide + (1 | Country) + (1 | ResponseId), data = df_z_control)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: score ~ Type * ide + (1 | Country) + (1 | ResponseId)
   Data: df_z_control

REML criterion at convergence: 23446.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4703 -0.7845  0.3707  0.7669  1.6509 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 0.08863  0.2977  
 Country    (Intercept) 0.02872  0.1695  
 Residual               0.87322  0.9345  
Number of obs: 8348, groups:  ResponseId, 4177; Country, 60

Fixed effects:
                 Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)     4.156e-01  4.405e-02  4.484e+02   9.434  < 2e-16 ***
TypeWEPTcc     -2.738e-01  4.957e-02  4.171e+03  -5.524 3.51e-08 ***
ide            -8.091e-03  6.512e-04  8.175e+03 -12.424  < 2e-16 ***
TypeWEPTcc:ide  7.592e-03  8.600e-04  4.171e+03   8.828  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation o

# 1.4 Mechanism

In [29]:
%Rpush df_control

In [30]:
%%R

M <- clmm(as.factor(WEPTcc) ~ ide*BELIEFcc + (1|Country), data = df_control, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ ide * BELIEFcc + (1 | Country)
data:    df_control

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 4208 -6635.50 13283.00 119(370) 1.11e+04 5.0e+08

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1        1       
Number of groups:  Country 60 

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
ide           1.120e-02  3.904e-03   2.869  0.00411 ** 
BELIEFcc      1.711e-02  2.951e-03   5.800 6.63e-09 ***
ide:BELIEFcc -1.200e-04  4.613e-05  -2.600  0.00931 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
            Estimate Std. Error z value
threshold.1 0.060483   0.286145   0.211
spacing     0.181225   0.004671  38.801
(878 observations deleted due to missingness)


In addition: Warning messages:
1: (2) Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables? 
In addition: Absolute and relative convergence criteria were met 
2: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 24
3: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 24


In [31]:
%%R

M <- clmm(as.factor(WEPTcc) ~ ide*POLICYcc + (1|Country), data = df_control, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ ide * POLICYcc + (1 | Country)
data:    df_control

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 4195 -6616.91 13245.83 121(379) 2.75e+03 4.1e+08

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1        1       
Number of groups:  Country 60 

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
ide           1.279e-02  4.180e-03   3.060  0.00222 ** 
POLICYcc      2.165e-02  3.504e-03   6.180 6.42e-10 ***
ide:POLICYcc -1.651e-04  5.567e-05  -2.966  0.00302 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
            Estimate Std. Error z value
threshold.1 0.198992   0.295401   0.674
spacing     0.181534   0.004682  38.770
(891 observations deleted due to missingness)


In addition: Warning messages:
1: (2) Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables? 
In addition: Absolute and relative convergence criteria were met 
2: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 24
3: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 24


# 2. How do climate interventions impact the conceptual and behavioral outcomes of people along the ideological spectrum?

## 2.1 Interventions’ Impact on Beliefs

In [17]:
df_belief = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'cond', 'ide','Ideology', "WEPTcc",'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'cond', 'ide','Ideology', 'WEPTcc'],
    var_name="Item",
    value_name="Belief"
)
df_belief['condName'] = df_belief['condName'].replace(['Control'], 'aControl')
df_belief = df_belief.dropna()
#df_belief.to_csv('df_belief.csv', index=False)

%Rpush df_belief

## TABLE S7: Belief by Ideology by Condition

In [33]:
%%R

M1 <- lmer(Belief ~ condName*Ideology + (1 | Country) + (1 | ResponseId), data = df_belief)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName * Ideology + (1 | Country) + (1 | ResponseId)
   Data: df_belief

REML criterion at convergence: 1686154

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.4167 -0.2762  0.0921  0.3015  6.0749 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 496.16   22.275  
 Country    (Intercept)  34.74    5.894  
 Residual               141.42   11.892  
Number of obs: 199111, groups:  ResponseId, 49824; Country, 60

Fixed effects:
                                           Estimate Std. Error         df
(Intercept)                                 76.6813     0.9190   117.1863
condNameBindingMoral                         0.8698     0.7155 49722.9215
condNameCollectAction                        2.6187     0.7122 49732.1280
condNameDynamicNorm                          0.4122     0.7133 49735.6248
condNameFutureSelfCont                       1


Correlation matrix not shown by default, as p = 24 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it

In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00221537 (tol = 0.002, component 1)


## TABLE S8: Policy by Ideology by Condition

In [18]:
df_policy = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'cond', 'Income', 'Edu', 'Age', 'Gender', 'ide','Ideology', "WEPTcc",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'cond', 'Income', 'Edu', 'Age', 'Gender', 'ide','Ideology', 'WEPTcc'],
    var_name="Item",
    value_name="Policy"
)
df_policy['condName'] = df_policy['condName'].replace(['Control'], 'aControl')
#df_policy.to_csv('df_policy.csv', index=False)

%Rpush df_policy

In [19]:
%%R

M1 <- lmer(Policy ~ condName*Ideology + (1 | Country) + (1 | ResponseId), data = df_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName * Ideology + (1 | Country) + (1 | ResponseId)
   Data: df_policy

REML criterion at convergence: 4142462

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.7254 -0.5002  0.1818  0.6138  3.4010 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 291.67   17.078  
 Country    (Intercept)  24.05    4.904  
 Residual               553.78   23.533  
Number of obs: 442975, groups:  ResponseId, 50509; Country, 60

Fixed effects:
                                           Estimate Std. Error         df
(Intercept)                               6.896e+01  7.608e-01  1.161e+02
condNameBindingMoral                      5.215e-01  5.861e-01  5.025e+04
condNameCollectAction                     2.716e+00  5.829e-01  5.023e+04
condNameDynamicNorm                       7.517e-01  5.841e-01  5.028e+04
condNameFutureSelfCont                    1.60


Correlation matrix not shown by default, as p = 24 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it



## TABLE S9: Action by Ideology by Condition

In [21]:
df_WEPT = df[["WEPTcc","ide", "Ideology", "Income", "Edu", "Age", "Gender", "Country", "condName", "cond", "condition_time_total"]]
df_WEPT['condName'] = df_WEPT['condName'].replace(['Control'], 'aControl')
#df_WEPT.to_csv('df_WEPT.csv', index=False)

%Rpush df_WEPT

In [22]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName*Ideology + (1|Country), data = df_WEPT, threshold = "equidistant")
summary(M)


Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName * Ideology + (1 | Country)
data:    df_WEPT

 link  threshold   nobs  logLik    AIC       niter       max.grad cond.H 
 logit equidistant 51186 -83144.09 166340.19 6360(25445) 3.40e+00 3.3e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2409   0.4908  
Number of groups:  Country 60 

Coefficients:
                                         Estimate Std. Error z value Pr(>|z|)
condNameBindingMoral                     -0.02784    0.05974  -0.466 0.641133
condNameCollectAction                    -0.14674    0.05930  -2.474 0.013346
condNameDynamicNorm                      -0.10060    0.05918  -1.700 0.089156
condNameFutureSelfCont                   -0.21300    0.06212  -3.429 0.000606
condNameLetterFutureGen                  -0.32356    0.06251  -5.176 2.26e-07
condNameNegativeEmotions                 -0.35812    0.05975  -5.994 2.05e-09
condNamePluralI

In [67]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName*Ideology + condition_time_total + (1|Country), data = df_WEPT, threshold = "equidistant")
summary(M)
#Table S10

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName * Ideology + condition_time_total +  
    (1 | Country)
data:    df_WEPT

 link  threshold   nobs  logLik    AIC       niter      max.grad cond.H 
 logit equidistant 51186 -82957.02 165968.05 1566(6260) 3.50e+04 4.1e+07

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1        1       
Number of groups:  Country 60 

Coefficients:
                                           Estimate Std. Error z value Pr(>|z|)
condNameBindingMoral                      1.903e-01  6.017e-02   3.163  0.00156
condNameCollectAction                    -1.314e-01  5.852e-02  -2.245  0.02478
condNameDynamicNorm                       4.592e-02  5.893e-02   0.779  0.43577
condNameFutureSelfCont                   -3.013e-01  6.187e-02  -4.870 1.12e-06
condNameLetterFutureGen                  -5.012e-01  6.257e-02  -8.009 1.16e-15
condNameNegativeEmotions                 -3.775e-01  5

In addition: Warning messages:
1: Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead. 
2: (2) Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables? 
In addition: Absolute and relative convergence criteria were met 
3: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 116
4: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 116


In [21]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName*condition_time_total + (1|Country), data = df_WEPT, threshold = "equidistant")
summary(M)
#Table S11

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName * condition_time_total + (1 | Country)
data:    df_WEPT

 link  threshold   nobs  logLik    AIC       niter      max.grad cond.H 
 logit equidistant 59392 -95191.20 190434.40 1439(5752) 4.41e+03 3.8e+07

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1        1       
Number of groups:  Country 63 

Coefficients:
                                                Estimate Std. Error z value
condNameBindingMoral                          -0.6875624  0.0622134 -11.052
condNameCollectAction                          0.1958999  0.0658078   2.977
condNameDynamicNorm                            0.4275081  0.0635497   6.727
condNameFutureSelfCont                         0.1648840  0.0682993   2.414
condNameLetterFutureGen                       -0.0208172  0.0729778  -0.285
condNameNegativeEmotions                       0.0598096  0.0666794   0.897
condNamePluralIgnor

In addition: Warning messages:
1: (2) Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables? 
In addition: Absolute and relative convergence criteria were met 
2: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 112
3: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 112


In [28]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName*condition_time_total*Ideology + (1|Country), data = df_WEPT, threshold = "equidistant")
summary(M)
#Table S16

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName * condition_time_total * Ideology +  
    (1 | Country)
data:    df_WEPT

 link  threshold   nobs  logLik    AIC       niter       max.grad cond.H 
 logit equidistant 51186 -81366.24 162832.47 5243(20968) 7.32e+03 3.6e+08

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1        1       
Number of groups:  Country 60 

Coefficients:
                                                                Estimate
condNameBindingMoral                                          -7.575e-01
condNameCollectAction                                          1.665e-01
condNameDynamicNorm                                            3.687e-01
condNameFutureSelfCont                                         1.395e-01
condNameLetterFutureGen                                       -1.254e-01
condNameNegativeEmotions                                       7.281e-02
condNamePluralIgnoran

condNameSciConsens                                              0.921 0.356801
condNameSystemJust                                              1.874 0.060888
condNameWorkTogetherNorm                                        0.937 0.348843
condition_time_total                                            5.875 4.22e-09
IdeologyLiberal                                                -0.731 0.464987
condNameBindingMoral:condition_time_total                      24.459  < 2e-16
condNameCollectAction:condition_time_total                     -2.897 0.003771
condNameDynamicNorm:condition_time_total                       -2.533 0.011305
condNameFutureSelfCont:condition_time_total                    -4.283 1.84e-05
condNameLetterFutureGen:condition_time_total                   -3.731 0.000190
condNameNegativeEmotions:condition_time_total                  -4.499 6.84e-06
condNamePluralIgnorance:condition_time_total                   -4.391 1.13e-05
condNamePsychDistance:condition_time_total          

In addition: Warning messages:
1: Using formula(x) is deprecated when x is a character vector of length > 1.
  Consider formula(paste(x, collapse = " ")) instead. 
2: (2) Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables? 
In addition: Absolute and relative convergence criteria were met 
3: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 208
4: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 208


In [23]:
dfc_share = df_control[["SHAREcc", "BELIEFcc", "Ideology", "Income", "Edu", "Age", "Gender", "Country"]]
#dfc_share.to_csv('dfc_share.csv', index=False)

%Rpush dfc_share

In [25]:
df_share = df[["SHAREcc","ide", "Ideology", "Income", "Edu", "Age", "Gender", "Country", "condName", "cond", "condition_time_total"]]
df_share['condName'] = df_share['condName'].replace(['Control'], 'aControl')
#df_share.to_csv('df_share.csv', index=False)

%Rpush df_share

In [26]:
%%R

M <- glmer(SHAREcc ~ condName*Ideology + (1|Country), data = df_share, family=binomial())
summary(M)
#Table S15

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName * Ideology + (1 | Country)
   Data: df_share

     AIC      BIC   logLik deviance df.resid 
 45313.1  45526.8 -22631.6  45263.1    38099 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.6658 -0.7742  0.3899  0.7631  2.6144 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1.164    1.079   
Number of obs: 38124, groups:  Country, 60

Fixed effects:
                                         Estimate Std. Error z value Pr(>|z|)
(Intercept)                               0.26925    0.15042   1.790 0.073460
condNameBindingMoral                      0.18178    0.07709   2.358 0.018368
condNameCollectAction                     0.25502    0.07800   3.270 0.001077
condNameDynamicNorm                       0.25419    0.07777   3.268 0.001082
condNameFutureSelfCont                    0.11461    0.08173   1.


Correlation matrix not shown by default, as p = 24 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it



In [29]:
df_Lib_share = df.query('Ideology == "Liberal"')
df_Con_share = df.query('Ideology == "Conservative"')

df_Lib_share['condName'] = df_Lib_share['condName'].replace(['Control'], 'aControl')
df_Con_share['condName'] = df_Con_share['condName'].replace(['Control'], 'aControl')

%Rpush df_Lib_share
%Rpush df_Con_share


In [30]:
%%R

M <- glmer(SHAREcc ~ condName + (1|Country), data = df_Lib_share, family=binomial())
summary(M)
#Table S13

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName + (1 | Country)
   Data: df_Lib_share

     AIC      BIC   logLik deviance df.resid 
 23327.4  23429.6 -11650.7  23301.4    19181 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2102 -0.7771  0.4045  0.7946  2.6114 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1.047    1.023   
Number of obs: 19194, groups:  Country, 60

Fixed effects:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -0.16085    0.14338  -1.122  0.26191    
condNameBindingMoral      0.19889    0.07596   2.618  0.00883 ** 
condNameCollectAction     0.51613    0.07763   6.648 2.97e-11 ***
condNameDynamicNorm       0.32230    0.07616   4.232 2.32e-05 ***
condNameFutureSelfCont    0.47986    0.08135   5.899 3.66e-09 ***
condNameLetterFutureGen   0.46067    0.08245   5.587 2.31e-08 ***


In [31]:
%%R

M <- glmer(SHAREcc ~ condName + (1|Country), data = df_Con_share, family=binomial())
summary(M)
#Table S14

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName + (1 | Country)
   Data: df_Con_share

     AIC      BIC   logLik deviance df.resid 
 21951.4  22053.4 -10962.7  21925.4    18917 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9469 -0.7464  0.3423  0.7447  2.5011 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1.417    1.191   
Number of obs: 18930, groups:  Country, 60

Fixed effects:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)               0.31149    0.16391   1.900  0.05738 .  
condNameBindingMoral      0.18613    0.07815   2.382  0.01724 *  
condNameCollectAction     0.25877    0.07902   3.275  0.00106 ** 
condNameDynamicNorm       0.25778    0.07880   3.271  0.00107 ** 
condNameFutureSelfCont    0.12089    0.08290   1.458  0.14477    
condNameLetterFutureGen   0.34477    0.08463   4.074 4.62e-05 ***


In [27]:
# Table S12

In [15]:
%%R

M1 <- lmer(Belief ~ ide + Age + ide*Age + (1|Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ ide + Age + ide * Age + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 127600.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2303 -0.2819  0.0909  0.3207  5.8733 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 496.9332 22.2920 
 Country    (Intercept)  25.1932  5.0193 
 Item       (Intercept)   0.2647  0.5145 
 Residual               148.6218 12.1911 
Number of obs: 14990, groups:  ResponseId, 3750; Country, 60; Item, 4

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)  7.984e+01  2.582e+00  2.109e+03  30.924  < 2e-16 ***
ide          7.362e-02  4.335e-02  3.720e+03   1.698   0.0895 .  
Age          2.406e-01  5.829e-02  3.739e+03   4.129 3.73e-05 ***
ide:Age     -6.430e-03  9.978e-04  3.745e+03  -6.445 1.31e-10 ***
---
Signif. co

In addition: Warning messages:
1: Some predictor variables are on very different scales: consider rescaling 
2: Some predictor variables are on very different scales: consider rescaling 


In [16]:
%%R

M1 <- lmer(Policy ~ ide + Age + ide*Age + (1|Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ ide + Age + ide * Age + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 299013.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5833 -0.5264  0.0623  0.6271  4.2154 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 312.85   17.69   
 Country    (Intercept)  17.73    4.21   
 Item       (Intercept) 147.07   12.13   
 Residual               407.27   20.18   
Number of obs: 32915, groups:  ResponseId, 3742; Country, 60; Item, 9

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)  6.679e+01  4.562e+00  1.294e+01  14.643 1.97e-09 ***
ide          8.595e-02  3.560e-02  3.708e+03   2.414   0.0158 *  
Age          2.339e-01  4.788e-02  3.724e+03   4.884 1.08e-06 ***
ide:Age     -4.852e-03  8.192e-04  3.729e+03  -5.923 3.46e-09 ***
---
Signif. co

In addition: Warning messages:
1: Some predictor variables are on very different scales: consider rescaling 
2: Some predictor variables are on very different scales: consider rescaling 


In [14]:
%%R

M1 <- clmm(as.factor(WEPTcc) ~ ide + Age + ide*Age + (1|Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
print(summary(M1))

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ ide + Age + ide * Age + (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 4207 -6586.05 13184.10 120(364) 1.12e+03 9.8e+07

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1        1       
Number of groups:  Country 60 

Coefficients:
          Estimate Std. Error z value Pr(>|z|)    
ide     -4.113e-03  3.377e-03  -1.218    0.223    
Age      2.398e-02  4.788e-03   5.009 5.49e-07 ***
ide:Age  6.709e-05  8.211e-05   0.817    0.414    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -0.573690   0.232906  -2.463
spacing      0.186292   0.004799  38.819
(879 observations deleted due to missingness)


In addition: Warning messages:
1: (2) Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables? 
In addition: Absolute and relative convergence criteria were met 
2: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 24
3: In update.uC(rho) : Non finite negative log-likelihood
  at iteration 24
